In [1]:
import numpy as np
import itertools as it
import pandas as ps

In [2]:
# "global" constants
rels = ["friend", "enemy"]

all_games = dict(
    pd = [[8, 8], # Result 1
          [0,12], # 2
          [12,0], # 3
          [4, 4]] # 4
    ,
    threat = [[12,6],
              [6,12],
              [6,0],
              [0,6]]
    ,
    disjunctive = [[12,12],
                   [12,12],
                   [12,12],
                   [0,0]]
    ,
    coordination = [[12,12],
                  [ 0, 0],
                  [ 0, 0],
                  [12,12]]
    ,
    singleControl = [[6, 6],
                  [0, 6],
                  [6, 6],
                  [0, 6]]
)

test_game = all_games["pd"]
# test_game

In [3]:
list(all_games.keys())

['threat', 'disjunctive', 'singleControl', 'pd', 'coordination']

In [4]:
def quick_format(s):
    return np.around(s, decimals=3).tolist()

In [5]:
def switch_player(player):
    if player.lower() == "a":
        return "b";
    elif player.lower() == "b":
        return "a";
    else:
        raise ValueError("Was not a player 'a' or 'b'!")

In [6]:
# total utility
def calculate_TU(result, w1, w2, gb, player):
    u1 = result[0]
    u2 = result[1]
    
    # alternate perspectives
    if player.lower() == "a":
        return (w1 * u1) + (gb * w2 * u2);
    elif player.lower() == "b":
        return (w1 * u2) + (gb * w2 * u1); # original
    else:
        raise ValueError("Was not a player 'a' or 'b'!")

In [8]:
# expected utility
def calculate_EU(u1, u2, p1=0.5, p2=0.5):
    return (u1 * p1) + (u2 * p2); # p: probs of choices

In [9]:
def calculate_probs(eu_choice, eu_other): # probability of a choice
    return np.exp(eu_choice) / (np.exp(eu_choice) + np.exp(eu_other))

In [10]:
def normalize(probs):
    total = np.sum(probs)
    ret_res = []
    for e in probs:
        ret_res.append(e / total)
    
    return ret_res;

In [12]:
def predict(all_tu, player):  # "innermost" call in recursion
    all_eu = [] # keeps default p=0.5 values - see calculate_EU
    if player.lower() == "a":
        all_eu.append(calculate_EU(all_tu[0], all_tu[1])) #a1
        all_eu.append(calculate_EU(all_tu[2], all_tu[3])) #a2
    elif player.lower() == "b":
        all_eu.append(calculate_EU(all_tu[0], all_tu[2])) #b1
        all_eu.append(calculate_EU(all_tu[1], all_tu[3])) #b2
    else:
        raise ValueError("Was not a player 'a' or 'b'!")
    
    probs = []
    probs.append(calculate_probs(all_eu[0], all_eu[1])) # a1 or b1
    probs.append(calculate_probs(all_eu[1], all_eu[0])) # a2 or b2
    
    return normalize(probs);

In [13]:
def predict_recursive(game, w1, w2, gb, player): # player is current level i.e. A (me) simulating B -> player=B
    all_tu = [] # array of total utilities for player in each result
    for result in game:
        all_tu.append(calculate_TU(result, w1, w2, gb, player))
    
    p = predict(all_tu, switch_player(player)) # probs based on p1 = p2 = 0.5
    
    all_eu = []
    if player.lower() == "a":
        all_eu.append(calculate_EU(all_tu[0], all_tu[1], p[0], p[1])) #a1
        all_eu.append(calculate_EU(all_tu[2], all_tu[3], p[0], p[1])) #a2
    elif player.lower() == "b":
        all_eu.append(calculate_EU(all_tu[0], all_tu[2], p[0], p[1])) #b1
        all_eu.append(calculate_EU(all_tu[1], all_tu[3], p[0], p[1])) #b2
    
    probs = []
    probs.append(calculate_probs(all_eu[0], all_eu[1])) # a1 or b1
    probs.append(calculate_probs(all_eu[1], all_eu[0])) # a2 or b2
    
    return normalize(probs);

In [14]:
def get_choice_probs(game, relationship, player):
    start = 51
    end = 101
    gb = 1 # default settings for "stranger"
    if (relationship == "friend"):
        start = 0
        end = 50
    elif (relationship == "enemy"):
        gb = -1
    
    outcomes = []
    for i in range(start, end):
        w1 = i / 100 # counting by 0.01
        w2 = 1 - w1
        outcomes.append(predict_recursive(game, w1, w2, gb, player)) # ex: returns simulation of player B
    
    final = np.mean(outcomes, axis=0)
#     print("Probability of choice {}1: {}".format(player.upper(), final[0]))
#     print("Probability of choice {}2: {}".format(player.upper(), final[1]))
    return final;

In [15]:
get_choice_probs(test_game, "friend", "a")

array([ 0.98006898,  0.01993102])

In [16]:
def get_outcome_probs(game, relationship):
    player = "a";
    other_p = switch_player(player)
    probs = get_choice_probs(game, relationship, player).tolist()
    
    probs.extend(get_choice_probs(game, relationship, other_p).tolist())
    
    ls = list(it.product(probs[0:2],probs[2:4]))

    res = []
    for e in ls:
        res.append(e[0]*e[1])
#     print(res)
    
#     final = np.around(res, decimals=3)
#     for i in range(4):
#         print("Probability of outcome {}: {}".format(game[i], final[i]))
    return res;

In [26]:
get_outcome_probs(all_games["singleControl"], "friend")

[0.4925373530947524,
 0.007462646905247512,
 0.4925373530947524,
 0.007462646905247512]

In [18]:
# returns P(r|o) for one relationship
def get_rel_prob(game, outcome, relationship): # outcome is index in game
    prob_sum = 0
    o_prob = 0

    for r in rels:
        p = get_outcome_probs(game, r)[outcome]
#         print(p)
        prob_sum += p * (1/2)
#         print(prob_sum)
        if r == relationship:
            o_prob = p
            
    return (o_prob * (1/2)) / prob_sum;

In [19]:
# returns P(f|o), P(e|o) for outcome in that order
def get_all_rel_probs(game, outcome):
    rel_probs = []
    for r in rels:
        rel_probs.append(get_rel_prob(game, outcome, r))
        
    return normalize(rel_probs);

In [20]:
# returns P(f|o), P(e|o) for each outcome in game in that order
def get_all_combos(game):
    final = []
    for o in range(len(game)):
        final.append(get_all_rel_probs(game, o))
        
    return final;

In [21]:
quick_format(get_all_rel_probs(test_game, 0)) # result: [friends, enemies]

[1.0, 0.0]

In [22]:
quick_format(get_all_combos(test_game))

[[1.0, 0.0], [0.711, 0.289], [0.711, 0.289], [0.0, 1.0]]

In [23]:
def generate_game_predictions():
    mp = []
    current = []
    current_p = []
    for game in all_games.keys():
        for o in range(4): # 4 cells per game
            current = []
            current.append(game + str(o))
            current_p = get_all_rel_probs(all_games[game], o)
            for r in range(2): # 2 relationships
                current.append(rels[r])
                current.append(quick_format(current_p[r]))
                mp.append(current)
                current = current[:1]

    predictions = ps.DataFrame(data=mp, columns=["game","relationship","rating"])
    remove_these = ["pd2","disjunctive1","disjunctive2","coordination2","coordination3",
                   "singleControl2","singleControl3"]
    predictions = predictions.loc[~predictions['game'].isin(remove_these)]
    
    predictions.to_csv("NEWmodelPredictions_noS.csv",index=False)
    return;

In [24]:
# generate_game_predictions()